In [ ]:
import bw2data as bd
import pandas as pd
from consumption_model_ch.utils import get_habe_filepath
import numpy as np

In [ ]:
def add_consumption_all_hh(
        co_name,
        dir_habe=None,
        option='disaggregated',
        write_dir="write_files",
):
    # 1. Get some metadata from the consumption database
    co = bd.Database(co_name)
    year_habe = co.metadata['year_habe']
    dir_habe = dir_habe or co.metadata['dir_habe']

    # 2. Extract total demand from HABE
    path_beschrei = get_habe_filepath(dir_habe, year_habe, 'Datenbeschreibung')
    path_ausgaben = get_habe_filepath(dir_habe, year_habe, 'Ausgaben')
    path_mengen = get_habe_filepath(dir_habe, year_habe, 'Mengen')

    # change codes to be consistent with consumption database and Andi's codes
    co = bd.Database(co_name)
    ausgaben = pd.read_csv(path_ausgaben, sep='\t')
    mengen = pd.read_csv(path_mengen, sep='\t')
    ausgaben.columns = [col.lower() for col in ausgaben.columns]
    mengen.columns = [col.lower() for col in mengen.columns]
    codes_co_db = sorted([act['code'] for act in co])
    columns_a = ausgaben.columns.values
    columns_m = [columns_a[0]]
    for code_a in columns_a[1:]:
        code_m = code_a.replace('a', 'm')
        if code_m in codes_co_db:
            columns_m.append(code_m)
        else:
            columns_m.append(code_a)
    ausgaben.columns = columns_m

    # Compute total consumption
    total_consumption = ausgaben.sum()
    total_consumption = total_consumption.drop('haushaltid')
    mengen = mengen.sum()
    mengen = mengen.drop('haushaltid')
    for i in range(len(mengen)):
        try:
            total_consumption[mengen.index[i]] = mengen.values[i]
        except KeyError:
            print(mengen.index[i])

    # Add other useful info, eg number of households and number of people
    meta = pd.read_excel(path_beschrei, sheet_name='Tabellen', skiprows=8, usecols=[0, 1, 3, 4])
    meta.columns = ['category1', 'category2', 'n_rows', 'n_cols']
    meta.dropna(subset=['n_rows'], inplace=True)

    # Combine some columns together
    temp1 = meta[meta['category1'].notnull()][['category1', 'n_rows', 'n_cols']]
    temp1.columns = ['category2', 'n_rows', 'n_cols']
    temp2 = meta[meta['category2'].notnull()][['category2', 'n_rows', 'n_cols']]
    meta = pd.concat([temp1, temp2])
    meta.set_index('category2', inplace=True)

    # Add info
    total_consumption['n_households'] = meta.loc['HABE{}_Ausgaben'.format(year_habe)]['n_rows']
    total_consumption['n_people'] = meta.loc['HABE{}_Personen'.format(year_habe)]['n_rows']

    # Save total demand
    write_dir = Path(write_dir)
    path_demand = write_dir / "habe_totaldemands.xlsx"
    total_consumption.to_excel(path_demand)

    # 3. Options

    # OPTION 1 aggregated. Total demands extract directly from HABE raw files
    # Excel file `habe_totaldemands.xlsx` contains sums of all private households in Switzerland for all categories of
    # the HBS. Units are the same as in the HBS (please refer to the SI-excel of Andi's ES&T-paper in order to translate
    # the codenames). The attached vector is in "per month" quantities.

    # OPTION 2 disaggregated. Andi's total demands from his Swiss consumption model
    # Excel file `heia2_totaldemands.xlsx` contains sums of all private households in Switzerland for all categories of
    # the HBS. Please note that the units are basically the same as in the HBS (please refer to the SI-excel of Andi's
    # ES&T-paper in order to translate the codenames). However, the attached vector is in "per year" instead of in
    # "per month". Furthermore, there are a couple of demands that were computed by the model itself. The codenames for
    # these computed/imputed categories start with "mx" and the units are as follows:
    # - kWh per year for electricity
    # - MJ per year for heating
    # - cubic meters per year for water supply and wastewater collection
    # - number of waste bags per year for refuse collection

    if option == 'aggregated':
        df = pd.read_excel(path_demand)
        df.columns = ['code', 'amount']
        df.set_index('code', inplace=True)
        n_households = int(df.loc['n_households', 'amount'])
        # n_people     = int(df.loc['n_people', 'amount'])
        df = df.drop(['n_households', 'n_people'])
        df = df.reset_index()

    elif option == 'disaggregated':
        path = dirpath / "functional_units" / 'habe20092011_hh_prepared_imputed.csv'
        df = pd.read_csv(path, low_memory=False)
        n_households = df.shape[0]
        df = df.drop('haushaltid', axis=1).sum()
        df = df.reset_index()
        df.columns = ['code', 'amount']

    else:
        n_households = None

    # 4. Add total inputs from Andi's model as swiss consumption activity
    co_act_name = 'ch hh all consumption {}'.format(option)
    try:
        co.get(co_act_name).delete()
    except:
        pass
    consumption_all = co.new_activity(co_act_name, name=co_act_name, location='CH', unit='1 month of consumption')
    consumption_all.save()
    # Add production exchange for the activity `consumption`
    consumption_all.new_exchange(
        input=(consumption_all['database'], consumption_all['code']),
        amount=1,
        type='production',
    ).save()
    consumption_all['agg_option'] = option
    consumption_all['n_households'] = n_households
    consumption_all.save()
    # Smth with codes
    codes = [act['code'] for act in co]
    unlinked_codes = []
    for i in range(len(df)):
        code = df.loc[i]['code']
        uncertainty_dict = get_uncertainty(code)
        if code in codes:
            consumption_all.new_exchange(
                input=(co.name, code),
                amount=df.loc[i]['amount'],
                type='technosphere',
                **uncertainty_dict,
                has_uncertainty=True,
            ).save()
        else:
            unlinked_codes.append(code)

    # Note that
    # - the number of consumption exchanges is the same as the number of activities in the database,
    # - but is a lot less than what Andi provided in his total demands. TODO not sure what this means anymore

In [ ]:
bd.projects.set_current("GSA for archetypes")
co_name = "swiss consumption 1.0"
co = bd.Database(co_name)

In [ ]:
a = co.random()
list(a.exchanges())[1].as_dict()

In [ ]:
data = ausgaben['a30'].values

In [ ]:
from scipy.stats import lognorm

In [ ]:
lognorm.fit??

In [ ]:
import plotly.graph_objects as go

In [ ]:
x = np.linspace(min(data), max(data), 10000)
pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
       / (x * sigma * np.sqrt(2 * np.pi)))

In [ ]:
num_bins = 200
bins_ = np.linspace(min(data), max(data), num_bins, endpoint=True)
freq, bins = np.histogram(data, bins=bins_)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=bins,
        y=freq,
        opacity=0.60,
        showlegend=True,
    ),
)

In [ ]:
bd.projects.set_current("GSA for archetypes")
co_name = "swiss consumption 1.0"

co = bd.Database(co_name)
year_habe = co.metadata['year_habe']
dir_habe = co.metadata['dir_habe']